In [1]:
import numpy as np
from skimage.transform import resize
import pylidc as pl
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)
n = scans.count()

In [3]:
image_size = (256, 256)

In [4]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    y = np.expand_dims(y, axis=-1)
    return X, y

In [5]:
model = UNet((*image_size, 1), batchnorm=True, dropout=False)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
for _ in range(25):
    i = np.random.randint(n)
    print(i)
    try:
        X, y = get_data(i)
        model.fit(X, y, batch_size=1, epochs=1)
    except OSError:
        print('Problem getting data')

341
Epoch 1/1
421/421 [==============================] - 218s 517ms/step - loss: 0.0032 - acc: 0.9997
740
Epoch 1/1
261/261 [==============================] - 82s 315ms/step - loss: 9.3846e-04 - acc: 1.0000
725
Epoch 1/1
259/259 [==============================] - 83s 322ms/step - loss: 6.4704e-04 - acc: 1.0000
308
Epoch 1/1
225/225 [==============================] - 72s 322ms/step - loss: 6.3052e-04 - acc: 1.0000
5
Epoch 1/1
112/112 [==============================] - 36s 318ms/step - loss: 0.0013 - acc: 0.9999
600
Epoch 1/1
267/267 [==============================] - 86s 321ms/step - loss: 4.2413e-04 - acc: 1.0000
409
Epoch 1/1
513/513 [==============================] - 164s 320ms/step - loss: 2.7651e-04 - acc: 1.0000
325
Epoch 1/1
278/278 [==============================] - 90s 323ms/step - loss: 0.0012 - acc: 0.9998
120
Epoch 1/1
133/133 [==============================] - 42s 319ms/step - loss: 6.7186e-04 - acc: 0.9999
886
Epoch 1/1
195/195 [==============================] - 62s 319ms/

In [8]:
model.save('unet.hdf5')